# This is a preview code!

In [2]:
import xarray as xr
import pygrib
import os

In [5]:
!ls data

air.2022.nc  uwnd.2022.nc  vwnd.2022.nc


In [ ]:
def dataInjection(source, target):
#     dirPaths = '/work/syseng/users/sdariza/workspace/grib-data'
    files = os.listdir(target)
    grbs = pygrib.open(target+'/'+files[0])
    grb = grbs.select(name='Temperature')
    lat,lon = grb.latlons()
    dataInterpolated, levelsNC, dates = interpolation(source, lat, lon)
    for f in files:
        for idx_time, i in enumerate(dates):
            if i in f:
                data = pygrib.open(dirPaths+'/'+f)
                grb = data.select(name='Temperature')
                for g in grb:
                    if g.level in levelsNC:
                        idx_level = np.where(levelsNC == g.level)
                        gridValuesLevelTime = dataInterpolated[idx_time,idx_level,:,:]
                        for i, value in enumerate(g.values):
                            for j in range(len(value)):
                                if gridValuesLevelTime[i][j] != 'nan':
                                    g.values[i][j] = gridValuesLevelTime[i][j]

                    

def interpolation(path, new_lat, new_lon):
    data = xr.open_dataset(path)
    dsi = data.interp(lat=new_lat, lon=new_lon, method='cubic')
    dates = ds.variables['time'].values
    datesFiltered = [str(i)[:13].replace('-','').replace('T','_') for i in dates]
    return dsi.to_array().values[0], data.variables['level'].values, datesFiltered
        